# Data Collector 
Для повышения модульности проект разбит на 2 блокнота - для сбора данных в датасет и собственно модель для работы с датасетом.  
Этот блокнот содержит сборщик данных

In [1]:
import numpy as np
import re
import pandas as pd
import glob
from html.parser import HTMLParser

import nltk
import nltk.tokenize as nt
import nltk.corpus as nc
import nltk.stem as ns
from nltk.stem import PorterStemmer,SnowballStemmer,LancasterStemmer
from nltk.tokenize.regexp import RegexpTokenizer

import pysrt

In [2]:
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Небольшой класс для разбора HTML, который встречается в субтитрах

In [3]:
class MyHTMLParser(HTMLParser):
    text =""
    def handle_starttag(self, tag, attrs):
        pass


    def handle_endtag(self, tag):
        pass


    def handle_data(self, data):
        self.text+=data



### Словарь

Так как операция создания словаря однократная , словари можно конвертировать из pdf в текст вручную
Поэтому Словарь загружается из конверированных в текст PDF , каждый файл содержит словарные статьи с указанием частей речи.  
Словарь должен содержать слова одной категории, категория указывается при загрузке файла
Загруженный словарь содержит для каждого слова 
* множество категорий grade (сейчас используется всегда максимальная )
* множество грамматических тэгов grammar ( сейчас не используется)

In [4]:
class Vocabulary():
#    part_of_speech = ['n.','v.','adj.','conj.','prep.','pron.']
#    template = parts_of_speech.join('')
    
    def __init__(self, filenames):
        self.words = {}
        for fn,grade in filenames.items():
            with open(fn) as f:
                for ln in f:
                    w,set_g = self.split_line(ln)
                    self.insert(word = w,grade = grade, grammar= set_g )
                            
    def split_line(self,ln):
        template='((n)|(v)|(prep)|(pron)|(conj)|(adj)|(adv))[\.\,]?'
        tokens =ln.split(' ')
        word = tokens[0]
        gram = set()
        for t in tokens[1:]:
            m = re.match(template,t) 
            if m:
                gram.add(m[1])
        return word,gram
                            
    def insert(self, word, grade, grammar):
        ww = self.words.pop(word,{})
        gr = ww.pop('grammar',set())
        gr |= grammar
        ww['grammar'] =  gr
        g = ww.pop('grade',set())
        g.add(grade)
        ww['grade']=g
        self.words[word] = ww 
        
    def summary(self):
        count= 0
        grades = {}
        grammars = {}
        for v in self.words.values():
            if not str(v['grammar']) in grammars:
                grammars[str(v['grammar'])] = 0 
            grammars[str(v['grammar'])]+=1
            if not str(v['grade']) in grades:
                grades[str(v['grade'])] = 0 
            grades[str(v['grade'])]+=1
            count+=1    
        return count,grades,grammars    


Простая обёртка для загрузки и чистки стоп-слов

In [5]:
class Stops:
    def __init__(self ):
        self.stop_words = set(nc.stopwords.words("english")) | {"'"}
    
    def purge(self,words): 
        return(  [ w for w in words if w.casefold() not in self.stop_words ] )

stops = Stops()    

Класс для сбора собственных имён, при анализе частота собственных имён является отдельным признаком 

In [6]:
class NE_Chunks:
    def __init__(self):
        pass 
    
    def extract(self,words):
        tags = nltk.pos_tag(words)
        tree = nltk.ne_chunk(tags, binary=True)
        ne =  set(
                " ".join(i[0] for i in t)
                for t in tree
                if hasattr(t, "label") and t.label() == "NE"
            )
        common = words      
        return(ne,common)
    
ne_chunks = NE_Chunks()    

На прикидках LancasterStemmer показал полное превосходство над остальными, не нашёл, в чём он может уступать

In [7]:
#stemmer = SnowballStemmer('english')
stemmer = LancasterStemmer()

### Сборщики статистики слов

Используются два класса для сбора статистики со всего файла  и с отдельного субтитра, они опираются на один базовый класс. Базовый сборщик подсчитывает количество слов каждого уровня(A1-С1) , количество собственных имен и количество всех слов во фрагменте, а также продолжительность фрагмента.  
Для каждого уровня  подсчитываются как общее количество слов, так и количество раздельных слов в своем фрагменте текста   
К сожалению,  порядка 30 % слов не попадает в подсчет (баг пока не найден).

In [8]:
class SubHandler:
    
    def tokenize(self,text):
        rt = RegexpTokenizer(r"\w+|\'")
        return( rt.tokenize(text) ) 
    
    def parse(self):
        self.tokens = stops.purge( self.tokenize(self.text) )
        self.ne,self.tokens =  ne_chunks.extract(self.tokens)
        self.tokens = [w.lower() for w in self.tokens ]
        self.ne = {w.lower() for w in self.ne }           # ---> Named Entities
        self.summary = nltk.FreqDist(self.tokens)         # ---> count entries for each word
    
    def classify(self,vocab):
        def find_word_grade(w):                    # find a word in original or stemmed form and report its grade
            gr = ''
            try:
                if w in vocab.words  : 
                    gr = max(vocab.words[w]['grade'])
                elif  (stemmer.stem(w) in vocab.words):    
                    gr= max(vocab.words[stemmer.stem(w)]['grade'])
            except ValueError as ve: 
                print(w,ve)
                
            return(gr) 
    
        def inc_dict_at( d,i,n):
            g = d.get(i,0)
            d[i] = g+n
  
                                      
        self.nums = dict()
        self.misses = {}
        
        g = ''
        for w,n in self.summary.items():
            # -- for each category , we accumulate general number of words with suffix _сnt
            # -- and number of different word with _wrd
            g = find_word_grade(w)
            if bool(g):
                inc_dict_at( self.nums,g+'_cnt',n)
                inc_dict_at( self.nums,g+'_wrd',1)
            elif w in self.ne:
                inc_dict_at( self.nums,'ne_cnt',n) 
            else:    
                self.misses[w]=n                                     
            inc_dict_at( self.nums,'count',n)
            inc_dict_at( self.nums,'length',len(w))
  
    def calc_freq(self):
        # this method fills statistic by fragment , that must be reported to upper level
        self.freq = self.nums.copy()
        # first we put numbers of words by category
        self.freq |= {'seconds':self.seconds()}
        wc = len( self.summary ) 
        # also frequences relative to time "_ps" and to amount of differnt words "_pw"
        self.freq |= { (k+'_pw'):round(v/wc,4) for (k,v) in self.nums.items() if k[-4:] in ['_cnt','_wrd'] }
        self.freq |= { (k+'_ps'):round(v/self.seconds(),2) for (k,v) in self.nums.items() if k[-4:] in ['_cnt','_wrd']}

    def statistic(self):
        self.calc_freq()
        return(self.freq )
 

#### Cборщик статистики по файлу
Из базового класса наследует все функции по сбору статистики
Кроме того, анализирует статистику по субтитрам, выбирая для каждого признака  средние и максимальные значения  

In [9]:
class Subfile(SubHandler):

    def __init__(self,filename):
        self.text = ''
        self.ne ={}
        self.freq = dict()
        self.summary = dict()
        self.tokens = list()
        self.misses = {}
        file_name_parts = filename.split('/')[-1].split('\\')[-1].split('.')
        self.movie = '.'.join( file_name_parts[:-1])                           # w/o 'srt'
        self.load(filename)
    
    def load(self,filename):
        encs = ['utf-8','iso-8859-1']
        last_error = None
        for en in encs:
            try:
                srt = pysrt.open(filename, encoding=en )
            except UnicodeDecodeError as e:
                last_error = e
                srt = []
            if srt :
                break
        
        if not srt:
            raise last_error 

        self.items = srt.read(filename)   
        
        h=MyHTMLParser()
        h.feed(srt.text)
        self.text = h.text  
# -----------------------
    def stat_by_of(self,agg,field):
    # aggregator to calculate max or sum->average for feature (field)    
        return( agg( st.get(field,0) for st in self.substatistics ) ) 
               
    def seconds(self): 
    # for file, duration is time between first and last subtitres    
        time_int = self.items[-1].end-self.items[0].start
        return ( 3600*time_int.hours + 60*time_int.minutes + time_int.seconds + round(time_int.milliseconds/1000,3) )
   

    def classify_all(self,vocab):

        self.substatistics = []               # -- storage to keep statistics for each subtitre
        for it in self.items:
            self.classify_one(it,vocab)
               
    def classify_one(self,it,vocab):  
        sb = Subtitle(it)
        if sb.seconds() <0.1:
            return
        sb.parse()
        sb.classify(vocab)
        st = sb.statistic()   
        st = self.st0 | st      
        self.substatistics.append(st)
 
    def classify(self,vocab):
    # first calucalate general stats as Subhandler, the analyze subtitres     
        super().classify(vocab)
        self.calc_freq()
        self.st0 = { e:0 for (e,v) in self.freq.items() }        
        self.classify_all(vocab)

        
    def statistic(self):
        ss = len(self.substatistics)
        maxs = { (k+'__max'):(self.stat_by_of(max,k)) for (k,v) in self.st0.items() } 
        means = { (k+'_mean'):(round(self.stat_by_of(sum,k)/ss,4)) for (k,v) in self.st0.items() }   
        return({'movie':self.movie} | maxs | means |  self.freq )

#### Сборщик статистики по субтитру

In [10]:
class Subtitle(SubHandler):
    def __init__(self,item):
        self.text = ''
        self.ne ={}
        self.freq = dict()
        self.summary = dict()
        self.tokens = list()
        self.misses = {}

        self.item = item
        self.text = item.text
        
    def seconds(self): 
        time_int = self.item.end-self.item.start
        return ( 3600*time_int.hours + 60*time_int.minutes + time_int.seconds + round(time_int.milliseconds/1000,3) )
       
    def statistic(self):
        self.calc_freq()
        return(self.freq)


#### Интегратор статистики по всем файлам
Сохраняет результаты в датасет в каталоге dataset

In [11]:
class DataCollector:

    def __init__(self): #, srt_pattern , scores_path)
#         self.srt_pattern= srt_pattern
#         self.xls_path = scores_path 
        self.process()
    
    def get_levels(self,xls_path):
        df = pd.read_excel(xls_path)
        df = df.drop('id',axis=1)
        df.columns = ['movie','level'] 
        resolve_levels = {'A2/A2+':'A2' ,'A2/A2+':'A2','A2/A2+, B1':'A2','B1, B2':'B1' }
        df['level'] = df.level.apply(lambda l: resolve_levels.get( l.strip(),l.strip() ) )
        return df

    def gather_stats(self,file_pattern,vocab):
        def add_one(acc, new ):
            for k in acc:
                if k not in new:
                    new[k]=0
            for k,v in new.items():
                if k in acc:
                    acc[k].append(v)
                else:
                    acc[k] = [v]

        acc = {}
        # ---- parse file , considering possible usincide error -------
        for fn in glob.glob(file_pattern):
            try:
                sf= Subfile(fn)        
                sf.parse()
            except UnicodeDecodeError as e:
                print(sf.movie,e)
                continue
            #  --- classify this and gather stats   
            sf.classify(vocab = vocab)
            st = sf.statistic()
            add_one(acc,st)
 
        return( pd.DataFrame.from_dict( acc ) ) 
    
    def join_data(self, ds , df ):
        dx = ds.merge(df,how='outer')
        return(dx)
        
    def write(self, dx ):
        dx.to_csv('datasets/movies.csv')

            
    def feature_cols(self,dx):
        return( [c for c in dx.columns if c != 'movie'] )
            
    def process(self):
        vocab =  Vocabulary({ 'Oxford_CEFR_level/A1.txt':'A1',
                       'Oxford_CEFR_level/A2.txt':'A2',
                      'Oxford_CEFR_level/B1.txt':'B1', 
                      'Oxford_CEFR_level/B2.txt':'B2',
                      'Oxford_CEFR_level/C1.txt':'C1'   } ) 
        df = self.get_levels('English_scores/movies_labels.xlsx')
        ds = self.gather_stats( 'English_scores/Subtitles_all/*/*.srt', vocab )
        dx = self.join_data(ds,df)
        self.write(dx)
         

### Run main program

In [12]:
%%time
DataCollector()

Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Downton Abbey - S01E07 - Episode 7.eng.SDH 'utf-8' codec can't decode byte 0xff in posit

#### TODO

* Найти причину утечки слов мимо словаря 
* Найти утечки фильмов при join
* Дать понятные наименования аттрибутам в Subhandle и производных классах
* Проверить возможность анализа грамматических тэгов в словаре и грамм.раборе  для более точного определения уроня слова
* Проводить очистку данных при выгрузке
** Ввести отладочный параметр для выгрузки датафрейма with_na/witout_na
** Находить корреляционные коллизии между признаками и удалять один из парв с корреляцией > 0.9